Using pandas to read the picke file for processing input sentence and correponding labels

In [12]:
import pandas as pd
corpus = pd.read_pickle('SST2.pkl')
sentences, labels = list(corpus.sentence), list(corpus.label)

In [3]:
print(len(sentences),len(labels))

79654 79654


Importing the relevant packages needed by the network.

In [13]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np


Generating the word frequency of each word

In [14]:
max_len = 0
word_freqs = collections.Counter()
num_recs = len(sentences)
label_list=[]
for index,sentence in enumerate(sentences):
    label_list.append(labels[index])
    words=nltk.word_tokenize(sentence.lower())
    if(len(words)>max_len):
        max_len=len(words)
    for word in words:
        word_freqs[word]+=1
nos_classes=len(list(set(labels)))
label_set_list=list(set(labels))
MAX_FEATURES=7000
MAX_SENTENCE_LENGTH=80



Generating the vocabullary for the embedding layer and word2index dictionary

In [15]:
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word2index = {x[0]: i+2 for i, x in
enumerate(word_freqs.most_common(MAX_FEATURES))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

Generating the input and output tensor for the model

In [16]:
X = np.empty((num_recs, ), dtype=list)
nos_classes=2
y = np.zeros((num_recs, nos_classes))
i = 0
# f_train = open(os.path.join(current_dir, file_name), 'rb')
for index,sentence in enumerate(sentences):
    l=[0]*nos_classes
    seqs=[]
    words=nltk.word_tokenize(sentence.lower())
    for word in words:
        if word in word2index.keys():
            seqs.append(word2index[word])
        else:
            seqs.append(word2index['UNK'])
    X[i]=seqs
    l[label_set_list.index(labels[index])]=1
    y[i]=l
    i+=1
X=sequence.pad_sequences(X,maxlen=MAX_SENTENCE_LENGTH)

Splting the dataset into training and testing sets

In [17]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2)
print(ytrain.shape)

(63723, 2)


Initializing the network model

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=MAX_SENTENCE_LENGTH))
model.add(Conv1D(filters=100, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=100, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Conv1D(filters=32, kernel_size=10, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 80, 32)            224064    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 80, 100)           32100     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 16, 100)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 16, 100)           100100    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 3, 100)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 3, 32)             32032     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1, 32)             0         
__________

Fitting the model.

In [19]:
NUM_EPOCHS=10
BATCH_SIZE=75
history=model.fit(Xtrain,ytrain,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,validation_data=(Xtest,ytest))

Train on 63723 samples, validate on 15931 samples
Epoch 1/10
63723/63723 [==============================] - 80s 1ms/step - loss: 0.3858 - acc: 0.8125 - val_loss: 0.2864 - val_acc: 0.8758
Epoch 2/10
63723/63723 [==============================] - 80s 1ms/step - loss: 0.2177 - acc: 0.9105 - val_loss: 0.2524 - val_acc: 0.8959
Epoch 3/10
63723/63723 [==============================] - 86s 1ms/step - loss: 0.1554 - acc: 0.9356 - val_loss: 0.2601 - val_acc: 0.9062
Epoch 4/10
63723/63723 [==============================] - 97s 2ms/step - loss: 0.1151 - acc: 0.9519 - val_loss: 0.2787 - val_acc: 0.8996
Epoch 5/10
63723/63723 [==============================] - 91s 1ms/step - loss: 0.0915 - acc: 0.9609 - val_loss: 0.3058 - val_acc: 0.9043
Epoch 6/10
63723/63723 [==============================] - 92s 1ms/step - loss: 0.0762 - acc: 0.9662 - val_loss: 0.3503 - val_acc: 0.9048
Epoch 7/10
63723/63723 [==============================] - 77s 1ms/step - loss: 0.0630 - acc: 0.9715 - val_loss: 0.3776 - val_acc

Plotting the accuracy and loss function for various iterations.

In [21]:
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="Train")
plt.plot(history.history["val_acc"], color="b", label="Validation")
plt.legend(loc="best")
plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="Train")
plt.plot(history.history["val_loss"], color="b", label="Validation")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

Testing with the new input text.

In [ ]:
new_X = np.empty((1, ), dtype=list)
test_sentence="this movie absolutely terrible ..not only was the acting awful but so was the sleep got while this movie played..this movie achieved the all powerful goal of crap ..i watched this movie thinking my 5$ wouldn't b in vein .but i was very wrong ..i guarantee u r better off just reading what i have to say about this unbelievably horrible movie b4 ..puttin yourself in the way of this dignity depriving movie ..i give it a negative 1 for trying..but no please no don't watch this movie..i had friends who joined me for this film..shortly after they were no longer found this movie actually will make u end your life please please don't see it i beg of u. if u see this movie make sure u destroy all copies because this movie is a spawn of Satan."
# test_sentence="""It's a strange feeling to sit alone in a theater occupied by parents and their rollicking kids. I felt like instead of a movie ticket, I should have been given a NAMBLA membership.<br /><br />Based upon Thomas Rockwell's respected Book, How To Eat Fried Worms starts like any children's story: moving to a new town. The new kid, fifth grader Billy Forrester was once popular, but has to start anew. Making friends is never easy, especially when the only prospect is Poindexter Adam. Or Erica, who at 4 1/2 feet, is a giant.<br /><br />Further complicating things is Joe the bully. His freckled face and sleeveless shirts are daunting. He antagonizes kids with the Death Ring: a Crackerjack ring that is rumored to kill you if you're punched with it. But not immediately. No, the death ring unleashes a poison that kills you in the eight grade.<br /><br />Joe and his axis of evil welcome Billy by smuggling a handful of slimy worms into his thermos. Once discovered, Billy plays it cool, swearing that he eats worms all the time. Then he throws them at Joe's face. Ewww! To win them over, Billy reluctantly bets that he can eat 10 worms. Fried, boiled, marinated in hot sauce, squashed and spread on a peanut butter sandwich. Each meal is dubbed an exotic name like the "Radioactive Slime Delight," in which the kids finally live out their dream of microwaving a living organism.<br /><br />If you've ever met me, you'll know that I have an uncontrollably hearty laugh. I felt like a creep erupting at a toddler whining that his "dilly dick" hurts. But Fried Worms is wonderfully disgusting. Like a G-rated Farrelly brothers film, it is both vomitous and delightful.<br /><br />Writer/director Bob Dolman is also a savvy storyteller. To raise the stakes the worms must be consumed by 7 pm. In addition Billy holds a dark secret: he has an ultra-sensitive stomach.<br /><br />Dolman also has a keen sense of perspective. With such accuracy, he draws on children's insecurities and tendency to exaggerate mundane dilemmas.<br /><br />If you were to hyperbolize this movie the way kids do their quandaries, you will see that it is essentially about war. Freedom-fighter and freedom-hater use pubescent boys as pawns in proxy wars, only to learn a valuable lesson in unity. International leaders can learn a thing or two about global peacekeeping from Fried Worms.<br /><br />At the end of the film, I was comforted when two chaperoning mothers behind me, looked at each other with befuddlement and agreed, "That was a great movie." Great, now I won't have to register myself in any lawful databases"""
test_sentence="""It's hard to say sometimes why exactly a film is so effective. From the moment I first came across "The Stone Boy", something told me it would be a great film. In spite of that, it seemed very unlikely that I'd ever have the opportunity to actually see it for myself. Then, one day, while looking through the online catalogue of my local library, I saw that they had recently purchased the DVD release of this film. Which I'm extremely glad for, because the cinematography is of a stunning depth and quality that an old VHS copy could never replicate.<br /><br />And speaking of the cinematography, I must single it out as far and above the most stunning aspect of this film. As a photographer who pursues very nearly the exact visual style portrayed in "The Stone Boy", I'm a firm believer in the fact that a great cinematographer can almost single-handedly carry a film. Here, he has a lot of help from an extremely talented cast, and a director who understands perfectly what the story needs. But to have Juan Ruiz Anchía behind the camera makes virtually every scene something of beauty. And you can almost never say that. Most films would never even expect such a thing of you. Scene after scene captures some detail, some little bit of visual magic that takes your breath away.<br /><br />The director, Christopher Cain, has had a long and interesting career. As far as I can gather, this film is not very representative of it. But, sometimes, to catch a director near the beginnings of his career, before all the big budgets and loss of focus, there's a real subtle magic to be found. Cain steps back in this film, lets things happen with a life of their own, and then ever further. Much like early John Sayles films, characters are given space to breathe, time to talk. Side stories happen because they do, and that's how life is. Cain displays a remarkable, raw, even outright painful understanding of human nature in this film.<br /><br />The acting ties much of this story together. When people talk, when they exist in this film, they do so as actual people, not held back by the fact that they are playing characters. Gina Berriault's script allows immensely talented and respected actors like Wilford Brimley, Robert Duvall, Glenn Close, and Frederic Forrest to spend time simply existing. Whether the things they have to say are minor or of deep significance, it all comes down with the weight of pure reality.<br /><br />When you look at the actors involved, or the great soundtrack by James Horner, it seems strange that such a film be very nearly forgotten. Maybe much of what makes "The Stone Boy" what it is was the time period it was made in. There's this 1970s hangover feeling to this picture that reminds me deeply of my own childhood. People talk of the 80s in terms of modern styles and music, but that's not the 80s I lived in or remember. The look of the images, the understated and dark knowing quality of the acting, and the overall result should get under the skin of any person who grew up in or near this era of time in North America. I see myself in this. I see how I saw the world. And a film like "The Stone Boy" sees the world for how it truly is.<br /><br />For more of this feeling, please see:<br /><br />The Black Stallion (1979), Never Cry Wolf (1983), Tender Mercies (1983), Testament (1983), Places in the Heart (1984), Matewan (1987), High Tide (1987), Driving Miss Daisy (1989), The Secret Garden (1993), The Secret of Roan Inish (1994), Wendy and Lucy (2008)"""
# test_sentence="""A typical romp through Cheech and Chong's reality which includes drugs, singing, more drugs, cars and driving, even more drugs, Pee Wee, aliens, gasoline, laundry, stand up comedy, surprisingly more drugs and SPACE COKE !!. It is not as coherent or plausible as Up in Smoke but it still is incredibly funny, without becoming as strange as Nice Dreams. There are some classic scenes, which include the opening scene where they get some gas for their car and the drive to work. Also funny is Cheech's song (Mexican-Americans) and Chong's follow up song. Another notable scene is the welfare office scene with Jones (human noise machine), from the Police Academy series, and the old laughing man. All in all, this is a great follow up to Up in Smoke and is quite watchable when sober or not.<br /><br />-Celluloid RehabDELIMETER1
# OK heres what I say: <br /><br />The movie was excellent. I am a huge Nancy fan and I have read all 1-56 original books and I went on to read more. I am now on 96. Beware of villains giving this movie a lower grade than it should have. All clues point to a wonderful movie! I loved the whole thing. So what Nancy is in current time. She is still old fashioned like she is in the books! People who haven't read more than 5 books are complaining about the view of Nancy. I have read all of them and I think Emma is perfect and that Nancy was perfect. I found parts of the movie spooky. I loved the exciting car chases and get aways. I loved the clues. I solved the mystery myself! It was really wonderful. I suggest you go see it since people who have been complaining know nothing of A what a good movie is and B about Nancy Drew. Go see it. It may not be Oscar worthy but its really a good movie"""
seqs=[]
test_sentence='most new movies have a bright sheen'
for word in test_sentence.split():
    if word not in stopwords_list:
        if word in word2index.keys():
            seqs.append(word2index[word])
        else:
            seqs.append(word2index['UNK'])
# print(seqs)
new_X[0]=seqs
new_X=sequence.pad_sequences(new_X,maxlen=MAX_SENTENCE_LENGTH)
y=model.predict(new_X)
label=label_list[np.argmax(y)]